## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [12]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [13]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [14]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1748194877654, experiment_id='0', last_update_time=1748194877654, lifecycle_stage='active', name='Default', tags={}>]

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/25 17:44:38 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/05/25 17:44:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/1e9a37872ad041cf9f8ea94893715e67/artifacts'
🏃 View run flawless-stork-744 at: http://127.0.0.1:5000/#/experiments/1/runs/1e9a37872ad041cf9f8ea94893715e67
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [16]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1748195078283, experiment_id='1', last_update_time=1748195078283, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1748194877654, experiment_id='0', last_update_time=1748194877654, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [17]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [18]:
client.search_registered_models()

[]

In [19]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/25 17:56:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748195770270, current_stage='None', description='', last_updated_timestamp=1748195770270, name='iris-classifier', run_id='1e9a37872ad041cf9f8ea94893715e67', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/1e9a37872ad041cf9f8ea94893715e67/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>